<a href="https://colab.research.google.com/github/AaronReichert/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [122]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [0]:
df['LAND_SQUARE_FEET']=(df['LAND_SQUARE_FEET'].str.replace(',',''))

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [126]:
# [ ] Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS'
df_one=df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']
df_one.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,3404,1328.0,1925.0,1,A1,0,01/01/2019
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,2042,1728.0,1935.0,1,A5,0,01/01/2019
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,2042,1728.0,1935.0,1,A5,0,01/01/2019
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,3333,1262.0,1925.0,1,A1,0,01/01/2019
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000,01/01/2019


In [127]:
# [ ] Use a subset of the data where the sale price was more than 100 thousand and less than 2 million.
df_high=df_one[df_one['SALE_PRICE']<2000000]
df_low=df_high[df_high['SALE_PRICE']>100000]
print(df_low.shape)
df_low.sample()

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
5747,2,OTHER,01 ONE FAMILY DWELLINGS,1,4939,116,NaN,A5,3873 AMUNDSON AVENUE,NaN,10466.0,1.0,0.0,1.0,1800,1872.0,1957.0,1,A5,230000,01/30/2019


In [128]:
# [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
df_date=df_low
df_date['SALE_DATE']=df_date['SALE_DATE'].astype('datetime64')
split = pd.to_datetime('2019-04-01')
df_train=df_low[df_low['SALE_DATE']<split]
df_test=df_low[df_low['SALE_DATE']>=split]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# I have no idea what that section of code is talking about


```
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
```



In [129]:
# double checking
df_train

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,1,A5,620000,2019-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,01 ONE FAMILY DWELLINGS,1,4081,44,NaN,A2,10 SEAFOAM STREET,NaN,10306.0,1.0,0.0,1.0,2400,921.0,1950.0,1,A2,330000,2019-03-29
18130,5,OTHER,01 ONE FAMILY DWELLINGS,1,2373,201,NaN,A5,74 MCVEIGH AVE,NaN,10314.0,1.0,0.0,1.0,2450,2128.0,1980.0,1,A5,690000,2019-03-29
18132,5,OTHER,01 ONE FAMILY DWELLINGS,1,1132,42,NaN,A1,479 VILLA AVENUE,NaN,10302.0,1.0,0.0,1.0,4361,1807.0,2018.0,1,A1,610949,2019-03-29
18134,5,OTHER,01 ONE FAMILY DWELLINGS,1,3395,37,NaN,A2,63 NUGENT AVENUE,NaN,10305.0,1.0,0.0,1.0,6000,621.0,1930.0,1,A2,520000,2019-03-29


In [130]:
df_test

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18235,2,OTHER,01 ONE FAMILY DWELLINGS,1,5913,878,NaN,A1,4616 INDEPENDENCE AVENUE,NaN,10471.0,1.0,0.0,1.0,5000,2272.0,1930.0,1,A1,895000,2019-04-01
18239,2,OTHER,01 ONE FAMILY DWELLINGS,1,5488,48,NaN,A2,558 ELLSWORTH AVENUE,NaN,10465.0,1.0,0.0,1.0,2500,720.0,1935.0,1,A2,253500,2019-04-01
18244,3,OTHER,01 ONE FAMILY DWELLINGS,1,5936,31,NaN,A1,16 BAY RIDGE PARKWAY,NaN,11209.0,1.0,0.0,1.0,2880,2210.0,1925.0,1,A1,1300000,2019-04-01
18280,3,OTHER,01 ONE FAMILY DWELLINGS,1,7813,24,NaN,A5,1247 EAST 40TH STREET,NaN,11210.0,1.0,0.0,1.0,1305,1520.0,1915.0,1,A5,789000,2019-04-01
18285,3,OTHER,01 ONE FAMILY DWELLINGS,1,8831,160,NaN,A9,2314 PLUMB 2ND STREET,NaN,11229.0,1.0,0.0,1.0,1800,840.0,1925.0,1,A9,525000,2019-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,01 ONE FAMILY DWELLINGS,1,13215,3,NaN,A2,244-15 135 AVENUE,NaN,11422.0,1.0,0.0,1.0,3300,1478.0,1925.0,1,A2,635000,2019-04-30
23031,4,OTHER,01 ONE FAMILY DWELLINGS,1,11612,73,NaN,A1,10919 132ND STREET,NaN,11420.0,1.0,0.0,1.0,2400,1280.0,1930.0,1,A1,514000,2019-04-30
23032,4,OTHER,01 ONE FAMILY DWELLINGS,1,11808,50,NaN,A0,135-24 122ND STREET,NaN,11420.0,1.0,0.0,1.0,4000,1333.0,1945.0,1,A0,635000,2019-04-30
23033,4,OTHER,01 ONE FAMILY DWELLINGS,1,12295,23,NaN,A1,134-34 157TH STREET,NaN,11434.0,1.0,0.0,1.0,2500,1020.0,1935.0,1,A1,545000,2019-04-30


In [131]:
# [ ] Do one-hot encoding of categorical features.
# first lets get info on the columns
colls=df_train.columns
print('unique values per column')
print('------------------------')
for col in colls:
  print (f'{col} ({df_low[col].nunique()})')

unique values per column
------------------------
BOROUGH (5)
NEIGHBORHOOD (6)
BUILDING_CLASS_CATEGORY (1)
TAX_CLASS_AT_PRESENT (2)
BLOCK (2496)
LOT (332)
EASE-MENT (0)
BUILDING_CLASS_AT_PRESENT (13)
ADDRESS (3135)
APARTMENT_NUMBER (1)
ZIP_CODE (125)
RESIDENTIAL_UNITS (3)
COMMERCIAL_UNITS (3)
TOTAL_UNITS (4)
LAND_SQUARE_FEET (1035)
GROSS_SQUARE_FEET (1050)
YEAR_BUILT (89)
TAX_CLASS_AT_TIME_OF_SALE (1)
BUILDING_CLASS_AT_TIME_OF_SALE (11)
SALE_PRICE (1000)
SALE_DATE (91)


In [132]:
# these should be 1 hot encoded
colls_hot=('BOROUGH','NEIGHBORHOOD','TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE')
for col in colls_hot:
  print (df_train[col].value_counts())
  print()

4    1204
5     662
3     398
2     242
1       1
Name: BOROUGH, dtype: int64

OTHER                 2382
FLUSHING-NORTH          77
FOREST HILLS            17
BOROUGH PARK            12
ASTORIA                 11
BEDFORD STUYVESANT       8
Name: NEIGHBORHOOD, dtype: int64

1     2476
1D      31
Name: TAX_CLASS_AT_PRESENT, dtype: int64

A1    919
A5    779
A2    411
A9    193
A0     67
S1     39
A3     38
A8     31
A6     14
A4     13
B2      1
B3      1
S0      1
Name: BUILDING_CLASS_AT_PRESENT, dtype: int64

A1    919
A5    779
A2    413
A9    193
A0     67
S1     39
A3     38
A8     31
A6     14
A4     13
S0      1
Name: BUILDING_CLASS_AT_TIME_OF_SALE, dtype: int64



In [133]:
# These should be left alone
colls=df_low.columns
colls_cold=colls.drop(['BOROUGH','NEIGHBORHOOD','TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE'])
for col in colls_cold:
  print (df_train[col].value_counts())
  print()

01 ONE FAMILY DWELLINGS    2507
Name: BUILDING_CLASS_CATEGORY, dtype: int64

16350    17
1272      7
5735      6
5506      5
6022      5
         ..
6732      1
593       1
2171      1
4697      1
12027     1
Name: BLOCK, Length: 2060, dtype: int64

20      46
1       45
19      43
7       40
13      38
        ..
220      1
222      1
226      1
228      1
1275     1
Name: LOT, Length: 304, dtype: int64

Series([], Name: EASE-MENT, dtype: int64)

104-18 187TH STREET          2
118-20 202ND STREET          2
57 CHESTNUT STREET           2
130-52 LEFFERTS BOULEVARD    2
117-45 125TH STREET          2
                            ..
216 BYRNE AVENUE             1
101-51 126TH STREET          1
26-23 210TH PLACE            1
185-30 FONDA AVENUE          1
514 EAST 29TH STREET         1
Name: ADDRESS, Length: 2497, dtype: int64

RP.    1
Name: APARTMENT_NUMBER, dtype: int64

10312.0    115
10306.0    113
10314.0     95
11434.0     77
11413.0     69
          ... 
10455.0      1
11102.0     

In [0]:
# now to do the actual one hot encoding
target='SALE_PRICE'
features=['BOROUGH','NEIGHBORHOOD','TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE']

In [0]:
# okay lets do the test train split
X_train=df_train[features]
y_train=df_train[target]
X_test=df_test[features]
y_test=df_test[target]

In [0]:
import category_encoders as ce

In [0]:
# this one hot encodes the chosen features
encoder=ce.one_hot.OneHotEncoder(use_cat_names=True)
X_train_enc=encoder.fit(X_train) #just to remember to not fit_transform both
X_train_enc=encoder.transform(X_train)
X_test_enc=encoder.transform(X_test)

In [0]:
# now lets add in the other columns
target='SALE_PRICE'
features=['BOROUGH','NEIGHBORHOOD','TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE']
colls=df_low.columns
dropped=features+[target]+['BUILDING_CLASS_CATEGORY','ADDRESS','APARTMENT_NUMBER','SALE_DATE','EASE-MENT','TAX_CLASS_AT_TIME_OF_SALE']
colls_merge=colls.drop(dropped)
X_train_merged=pd.concat([X_train_enc,df_train[colls_merge]],axis=1)
X_test_merged=pd.concat([X_test_enc,df_test[colls_merge]],axis=1)


# used this to double check
# print(X_train_enc.shape)
# print(df_train[colls_merge].shape)
# print(X_train_merged.shape)
# X_train_merged

In [0]:
# [ ] Do feature selection with SelectKBest.
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np


In [0]:
colls=X_train_merged.columns
for col in colls:
  X_train_merged[col]=X_train_merged[col].astype(np.float64)


In [0]:
X_train_merged

In [0]:
X_train_merged.info()

In [205]:
# how many to chose?
mae_train_list=[]
mae_test_list=[]
features=len(X_train_merged.columns)
x=0
while x < features:
  x+=1
  selector=SelectKBest(score_func=f_regression,k=x)
  X_train_kbest=selector.fit_transform(X_train_merged,y_train)
  X_test_kbest=selector.transform(X_test_merged)
  model=LinearRegression()
  model.fit(X_train_kbest,y_train)
  # x_pred=model.predict(X_train_kbest)
  y_pred=model.predict(X_test_kbest)
  # mae_train=mean_absolute_error(y_pred,y_test)
  mae_test=mean_absolute_error(y_pred,y_test)
  # mae_train.append(mae)
  print(f'MAE on test with {x} features ${mae_test:,.2f}')
  mae_test_list.append(mae_test)


MAE on test with 1 features $183,640.59
MAE on test with 2 features $179,554.77
MAE on test with 3 features $179,291.47
MAE on test with 4 features $179,291.47
MAE on test with 5 features $170,482.97
MAE on test with 6 features $169,982.37
MAE on test with 7 features $168,139.67
MAE on test with 8 features $168,244.54
MAE on test with 9 features $167,855.35
MAE on test with 10 features $164,737.04
MAE on test with 11 features $165,346.03
MAE on test with 12 features $164,860.23
MAE on test with 13 features $155,159.40
MAE on test with 14 features $156,541.35
MAE on test with 15 features $156,571.59
MAE on test with 16 features $156,572.86
MAE on test with 17 features $156,393.75
MAE on test with 18 features $156,393.75
MAE on test with 19 features $156,255.32
MAE on test with 20 features $156,255.32
MAE on test with 21 features $154,395.61
MAE on test with 22 features $154,425.71
MAE on test with 23 features $154,425.71
MAE on test with 24 features $154,425.71
MAE on test with 25 featu

In [206]:
mae_test_list

[183640.5858012459,
 179554.76843033519,
 179291.46658251373,
 179291.46658251635,
 170482.9711218082,
 169982.36848242593,
 168139.6729827943,
 168244.54060436052,
 167855.35127097488,
 164737.03811948252,
 165346.03301324803,
 164860.22749046236,
 155159.40462621563,
 156541.3479677845,
 156571.5945131868,
 156572.86128854705,
 156393.75380245,
 156393.7538024393,
 156255.31686588432,
 156255.31686587288,
 154395.60999133016,
 154425.7139719822,
 154425.7139719789,
 154425.71397196685,
 154425.7139719793,
 154121.52407649887,
 153987.11458783632,
 153911.35651237503,
 153862.73565796393,
 154713.0651562573,
 154838.93149347894,
 154838.93149348057,
 154787.79552005266,
 154781.24957733825,
 154760.23157892365,
 154592.60409115022,
 154592.60409115002,
 155529.08356766513,
 155696.90014274593,
 155763.94344240963,
 155747.01677730365,
 155979.10894246714,
 155942.8836794844,
 155939.15924067097,
 155918.72437888198,
 155937.9948078416]

In [187]:
X_train_merged['TAX_CLASS_AT_TIME_OF_SALE'].nunique()

1

In [185]:
X_train_merged

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5495.0,801.0,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1.0
61,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7918.0,72.0,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1.0
78,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4210.0,19.0,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1.0
108,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5212.0,69.0,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1.0
111,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7930.0,121.0,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4081.0,44.0,10306.0,1.0,0.0,1.0,2400.0,921.0,1950.0,1.0
18130,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2373.0,201.0,10314.0,1.0,0.0,1.0,2450.0,2128.0,1980.0,1.0
18132,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1132.0,42.0,10302.0,1.0,0.0,1.0,4361.0,1807.0,2018.0,1.0
18134,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3395.0,37.0,10305.0,1.0,0.0,1.0,6000.0,621.0,1930.0,1.0
